In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from IPython.display import display, HTML

In [9]:
# Doppler effect parameters

c = 343  # Speed of sound (m/s) in air

# Set up the figure and axis
fig, ax = plt.subplots()
plt.close()  # Prevent initial static plot from displaying

# Initial positions of source and observer
source_pos = 5  # Starting point of the wave source
observer_pos = 0  # Observer fixed at the origin (0, 0)

# Create the dot markers for the observer and the wave source
observer_dot, = ax.plot([], [], 'go', markersize=10)  # Green dot for observer
source_dot, = ax.plot([], [], 'ro', markersize=10)    # Red dot for the source


In [3]:
# Initialize plot limits and elements

def init():
    
    ax.set_xlim(-15, 15)
    ax.set_ylim(-15, 15)
    ax.set_aspect('equal')
    ax.set_title('Doppler Effect Visualization')
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')
    ax.legend(loc='upper right')
    
    observer_dot.set_data([observer_pos], [0])
    source_dot.set_data([source_pos], [0])
    
    return observer_dot, source_dot

In [4]:
# Function to draw circular waves

def draw_waves(source_x, frame):
    
    ax.patches = []  # Clear previous wave circles
    num_waves = 10  # Number of waves at a time
    max_radius = 10  # Maximum radius of waves

    # Calculate relative velocity and adjust wavelength
    relative_velocity = source_x - observer_pos
    wavelength_factor = 1 + relative_velocity / 20  # Adjust this factor for desired speed impact

    for i in range(num_waves):
        # Calculate the radius of each wave, expanding over time
        radius = (frame - i * 20) % max_radius
        adjusted_radius = radius * wavelength_factor

        # Only draw waves that are visible within the max_radius
        if adjusted_radius > 0:
            wave_color = 'blue' if relative_velocity < 0 else 'red'
            circle = plt.Circle((source_x, 0), adjusted_radius, color=wave_color, fill=False, lw=1.5)
            ax.add_patch(circle)

In [11]:
# Update function for the animation

def update(frame, source_x):
    source_dot.set_data([source_x], [0])
    draw_waves(source_x, frame)

    # Change the source dot color for redshift/blueshift
    if source_x < observer_pos:
        source_dot.set_color('blue')  # Blueshift (moving towards observer)
    else:
        source_dot.set_color('red')   # Redshift (moving away from observer)
    
    return observer_dot, source_dot

In [12]:
# Create interactive widgets

x_pos_slider = widgets.FloatSlider(min=-10, max=10, step=0.1, value=5, description="Source Position")
pause_button = widgets.ToggleButton(value=False, description="Pause Animation", button_style='info')

# Store the animation object to control play/pause
ani = None


In [13]:
# Function to handle the animation update interactively

def update_plot(source_x, is_paused):
    global ani

    # Start or pause the animation based on the button state
    if ani is not None:
        ani.event_source.stop()  # Stop any ongoing animation

    if not is_paused:
        ani = FuncAnimation(
            fig, update, frames=np.arange(0, 200), fargs=(source_x,),
            init_func=init, interval=50, blit=False, repeat=True
        )
        display(HTML(ani.to_jshtml()))

In [7]:
# Display the slider and attach the interactive functionality
ui = widgets.VBox([x_pos_slider])
display(ui)

# Connect the slider to the update function
widgets.interactive(update_plot, source_x=x_pos_slider)

interactive(children=(FloatSlider(value=5.0, description='Source Position', max=10.0, min=-10.0), Output()), _…